## Convert-Pheno
<em>A software toolkit for the interconversion of standard data models for phenotypic data.</em>


# Installation

*Note:* We'll perform an installation directly from the [GitHub](https://github.com/mrueda/convert-pheno) source.

First we will download the repository by using `git clone`

In [1]:
!git clone https://github.com/CNAG-Biomedical-Informatics/convert-pheno.git


Cloning into 'convert-pheno'...
remote: Enumerating objects: 3978, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 3978 (delta 84), reused 153 (delta 61), pack-reused 3772
Receiving objects: 100% (3978/3978), 67.30 MiB | 16.30 MiB/s, done.
Resolving deltas: 100% (2229/2229), done.
Updating files: 100% (128/128), done.


Now we enter into `convert-pheno` directory:

In [2]:
%cd convert-pheno/

/content/convert-pheno


And we perform the installation of the needed dependencies.
Note that this task will take **a few minutes** to complete...

In [3]:
!make -f makefile.install install

sudo apt-get install cpanminus libbz2-dev zlib1g-dev libperl-dev libssl-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libbz2-dev is already the newest version (1.0.8-5build1).
cpanminus is already the newest version (1.7045-1).
libperl-dev is already the newest version (5.34.0-3ubuntu1.2).
libssl-dev is already the newest version (3.0.2-0ubuntu1.10).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
cpanm --notest --installdeps .	
--> Working on .
Configuring /content/convert-pheno ... OK
<== Installed dependencies for .. Finishing.


Ok, finally let's run some tests to make sure all works as it should:

In [4]:
!make -f makefile.install test

prove -l
t/args.t ..... ok
t/cli.t ...... ok
t/mapping.t .. ok
t/module.t ... ok
t/ohdsi.t .... ok
t/stream.t ... ok
All tests successful.
Files=6, Tests=27, 21 wallclock secs ( 0.05 usr  0.01 sys + 18.30 cusr  0.89 csys = 19.25 CPU)
Result: PASS


The CLI is located at `bin/convert-pheno`. Let's execute it to visualize the help:

In [16]:
!bin/convert-pheno -h

Usage:
    convert-pheno [-i input-type] <infile> [-o output-type] <outfile>
    [-options]

         Arguments:                       
           (input-type): 
                 -ibff                    Beacon v2 Models ("individuals" JSON|YAML) file
                 -iomop                   OMOP-CDM CSV files or PostgreSQL dump
                 -ipxf                    Phenopacket v2 (JSON|YAML) file
                 -iredcap (experimental)  REDCap (raw data) export CSV file
                 -icdisc  (experimental)  CDISC-ODM v1 XML file

                 (Wish-list)
                 #-openehr                openEHR
                 #-ifhir                  HL7/FHIR

           (output-type):
                 -obff                    Beacon v2 Models ("invididuals" JSON|YAML) file
                 -opxf                    Phenopacket v2 (JSON|YAML) file

                 (Wish-list)
                 #-oomop                  OMOP-CDM PostgreSQL dump

         Options:
           -expo

# Command-line examples

Ok, we're going to run a few jobs. We'll be using the included test data.

### BFF to PXF

Our input file is going to be a Beacon v2 Models `individuals` file (BFF) in `json` format.

Let's take a look to its content with [JSON Crack](https://jsoncrack.com/editor?json=https://raw.githubusercontent.com/mrueda/convert-pheno/main/t/bff2pxf/in/individuals.json).

In [ ]:
#@title
%%html
<Iframe src="https://jsoncrack.com/editor?json=https://raw.githubusercontent.com/mrueda/convert-pheno/main/t/bff2pxf/in/individuals.json", width='1600', height='500'>

Nice!

The first job is going to be converting that file to Phenopackets v2 (PXF) in `json` format.

From now on. we'll be storing the files in the directory `/content`.

In [17]:
!bin/convert-pheno -ibff t/bff2pxf/in/individuals.json -opxf pxf.json -out-dir /content

Uhmm...no output on the screen, what happened?

Well, this is actually a sign that the program worked fine. No news is good news.

A new file was created at `/content/pxf.json`.

*Note:* You can access the directory `/content` by clicking in the folder icon on the left side navigation menu.

If you want to re-create a file and the script says the file exists you can use the option `-O`.

Let's use the Linux command `head` see the first 10 lines of the file.

In [19]:
!head /content/pxf.json

[
   {
      "diseases" : [],
      "id" : "phenopacket_id.GPkLruXtK",
      "measurements" : [
         {
            "assay" : {
               "id" : "LOINC:35925-4",
               "label" : "BMI"
            },


We will install the awesome utility [jq](https://stedolan.github.io/jq) to extract some information.

In [20]:
!apt install jq

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 357 kB of archives.
After this operation, 1,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libonig5 amd64 6.9.7.1-2build1 [172 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjq1 amd64 1.6-2.1ubuntu3 [133 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 jq amd64 1.6-2.1ubuntu3 [52.5 kB]
Fetched 357 kB in 1s (442 kB/s)
Selecting previously unselected package libonig5:amd64.
(Reading database ... 121979 files and directories currently installed.)
Preparing to unpack .../libonig5_6.9.7.1-2build1_amd64.deb ...
Unpacking libonig5:amd64 (6.9.7.1-2build1) ...
Selecting previously unselected package libjq1:am

Ok, the `[` indicate that the file is a JSON array. Now we we'll check the property `id` for all array elements.

In [21]:
!jq '.[].id' /content/pxf.json

"phenopacket_id.GPkLruXtK"
"phenopacket_id.FO4oFq6yc"
"phenopacket_id.hlOsfvE9z"
"phenopacket_id.eQJEawJog"
"phenopacket_id.eR66GvvrU"
"phenopacket_id.MyWvvlcVA"
"phenopacket_id.3JrlAsl8u"
"phenopacket_id.rHECRecEX"
"phenopacket_id.LQuuzc7z2"
"phenopacket_id.Fvt8cWY4T"


Great!

Ok, let's use `jq` again to fetch the `sex` property.

In [22]:
!jq '.[].subject.sex' /content/pxf.json

"MALE"
"FEMALE"
"FEMALE"
"FEMALE"
"MALE"
"FEMALE"
"MALE"
"MALE"
"FEMALE"
"MALE"


Let's complicate a bit the query. Let's find out the assay ontololgy for the 1st item inside `measurements`.

In [23]:
!jq '.[].measurements[0].assay.id' /content/pxf.json

"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"
"LOINC:35925-4"


If you want to learn more about `jq` I recommend you this [tutorial](https://sandbox.bio/tutorials?id=jq-intro&step=1) and this [playground](https://sandbox.bio/playgrounds?id=jq), both from [sandbox.io](https://sandbox.bio).

Another useful way of visualizing `json` data is by converting them to tables.

We'll do that with the help from Python's `pandas`.

The table that you will see is interactive. You can filter, sort, etc. and even save the data as `csv, json` and Markdown.

In [24]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Import json
df = pd.read_json('/content/pxf.json')
df.head()

,diseases,id,measurements,medicalActions,metaData,subject
0,[],phenopacket_id.GPkLruXtK,"[{'assay': {'id': 'LOINC:35925-4', 'label': 'B...","[{'procedure': {'code': {'id': 'OPCS4:L46.3', ...","{'created': '2023-08-30T15:48:27Z', 'createdBy...","{'id': 'HG00096', 'sex': 'MALE', 'vitalStatus'..."
1,[],phenopacket_id.FO4oFq6yc,"[{'assay': {'id': 'LOINC:35925-4', 'label': 'B...","[{'procedure': {'code': {'id': 'OPCS4:K06.1', ...","{'created': '2023-08-30T15:48:27Z', 'createdBy...","{'id': 'HG00097', 'sex': 'FEMALE', 'vitalStatu..."
2,[],phenopacket_id.hlOsfvE9z,"[{'assay': {'id': 'LOINC:35925-4', 'label': 'B...","[{'procedure': {'code': {'id': 'OPCS4:T89.4', ...","{'created': '2023-08-30T15:48:27Z', 'createdBy...","{'id': 'HG00099', 'sex': 'FEMALE', 'vitalStatu..."
3,[],phenopacket_id.eQJEawJog,"[{'assay': {'id': 'LOINC:35925-4', 'label': 'B...","[{'procedure': {'code': {'id': 'OPCS4:T77.2', ...","{'created': '2023-08-30T15:48:27Z', 'createdBy...","{'id': 'HG00100', 'sex': 'FEMALE', 'vitalStatu..."
4,[],phenopacket_id.eR66GvvrU,"[{'assay': {'id': 'LOINC:35925-4', 'label': 'B...","[{'procedure': {'code': {'id': 'OPCS4:Z01.4', ...","{'created': '2023-08-30T15:48:27Z', 'createdBy...","{'id': 'HG00101', 'sex': 'MALE', 'vitalStatus'..."


## OMOP to BFF

Now we're going to convert OMOP-CDM to BFF. This time we'll tell `Convert-Pheno` to be more **verbose**.

We'll be using the flag:

* `--verbose`

In [25]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals.json --out-dir /content --verbose

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.12             *
*   (C) 2022-2023 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <CONDITION_OCCURRENCE>
Rows total:     500

Processing table ... <DRUG_EXPOSURE>
Rows total:     500

Processing table ... <MEASUREMENT>
Rows total:     500

Processing table ... <OBSERVATION>
Rows total:     500

Processing table ... <PERSON>
Rows total:     500

Processing table ... <PROCEDURE_OCCURRENCE>
Rows total:     500

Processing table ... <VISIT_OCCURRENCE>
Rows total:     500

Individuals total:     500



Let's take a look to the data, just as we did with PXF. We'll be displaying only the 1st individual.

In [26]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Import json
df = pd.read_json('/content/omop_individuals.json')
df.head(1)

,diseases,ethnicity,geographicOrigin,id,info,interventionsOrProcedures,sex,phenotypicFeatures,treatments
0,[{'_info': {'CONDITION_OCCURRENCE': {'OMOP_col...,"{'id': 'NCIT:C41261', 'label': 'White'}","{'id': 'NCIT:C43857', 'label': 'Italian'}",1,{'PERSON': {'OMOP_columns': {'birth_datetime':...,[{'_info': {'PROCEDURE_OCCURRENCE': {'OMOP_col...,"{'id': 'NCIT:C20197', 'label': 'Male'}",NaN,NaN


Now we'll run `Convert-Pheno` again but we'll ask `Convert-Pheno` to print the SQL tables as `csv` files.

We'll be using the flag:

* `--sql2csv`

In [27]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals.json --out-dir /content --sql2csv --verbose -O

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.12             *
*   (C) 2022-2023 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <CONDITION_OCCURRENCE>
Rows total:     500

Processing table ... <DRUG_EXPOSURE>
Rows total:     500

Processing table ... <MEASUREMENT>
Rows total:     500

Processing table ... <OBSERVATION>
Rows total:     500

Processing table ... <PERSON>
Rows total:     500

Processing table ... <PROCEDURE_OCCURRENCE>
Rows total:     500

Processing table ... <VISIT_OCCURRENCE>
Rows total:     500

Individuals total:     500



And voila...now you have a bunch of `csv` files inside `/content`

The tables `PERSON` and `CONCEPT` are always read, but it is possible to process specific tables by using the option `--omop-tables`. Let's process only the table `DRUG_EXPOSURE`:


In [30]:
!bin/convert-pheno -iomop t/omop2bff/in/omop_cdm_eunomia.sql -obff omop_individuals_drug_exposure.json --omop-tables DRUG_EXPOSURE --out-dir /content --verbose

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.12             *
*   (C) 2022-2023 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <omop2bff> ====
Processing: <t/omop2bff/in/omop_cdm_eunomia.sql>
Writing </content/omop_individuals_drug_exposure.json> file

Processing table ... <CONCEPT>
Rows total:     444

Processing table ... <DRUG_EXPOSURE>
Rows total:     500

Processing table ... <PERSON>
Rows total:     500

Individuals total:     500



## REDCap to BFF

Finally we're going to convert REDCap to BFF. This conversion is more complex and it needs more input files.

Apart from the input `-iredcap` we'll be using the options:

* `--redcap-dictionary`
* `--mapping-file`

In [31]:
!bin/convert-pheno -iredcap t/redcap2bff/in/redcap_data.csv -obff redcap_individuals.json --redcap-dictionary t/redcap2bff/in/redcap_dictionary.csv --mapping-file  t/redcap2bff/in/redcap_mapping.yaml --out-dir /content --verbose

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.12             *
*   (C) 2022-2023 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <redcap2bff> ====
Processing: <t/redcap2bff/in/redcap_data.csv>
Writing </content/redcap_individuals.json> file



Awesome! Let's take a look to the data:




In [32]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Import json
df = pd.read_json('/content/redcap_individuals.json')
df.head(1)

,diseases,ethnicity,exposures,id,info,interventionsOrProcedures,measures,phenotypicFeatures,sex,treatments
0,[{'ageOfOnset': {'ageRange': {'end': {'iso8601...,"{'id': 'NCIT:C41261', 'label': 'Caucasian'}",[{'ageAtExposure': {'age': {'iso8601duration':...,107:week_0_arm_1,{'age': {'ageRange': {'end': {'iso8601duration...,[{'ageAtProcedure': {'age': {'iso8601duration'...,"[{'assayCode': {'id': 'NCIT:C51948', 'label': ...","[{'excluded': True, 'featureType': {'id': 'NCI...","{'id': 'NCIT:C20197', 'label': 'Male'}","[{'_info': {'dose': None, 'drug': 'budesonide'..."


Let's change some default parameters.

This time we'll perform a `mixed` search instead of an `exact` one (default).

We'll be using the option:

* `--search mixed`

In [33]:
!bin/convert-pheno -iredcap t/redcap2bff/in/redcap_data.csv -obff redcap_individuals.json --redcap-dictionary t/redcap2bff/in/redcap_dictionary.csv --mapping-file  t/redcap2bff/in/redcap_mapping.yaml --out-dir /content --search mixed --verbose -O

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.12             *
*   (C) 2022-2023 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <redcap2bff> ====
Processing: <t/redcap2bff/in/redcap_data.csv>
Writing </content/redcap_individuals.json> file



And now the same but changing the similarity method from `cosine` (default) to `dice` and setting the minimum similarity from `0.8` (default) to `0.5`.

We'll be using the options:

* `--search mixed`
* `--text-similarity-method dice`
* `--min-text-similarity-score 0.5`

In [34]:
!bin/convert-pheno -iredcap t/redcap2bff/in/redcap_data.csv -obff redcap_individuals.json --redcap-dictionary t/redcap2bff/in/redcap_dictionary.csv --mapping-file  t/redcap2bff/in/redcap_mapping.yaml --out-dir /content --search mixed -text-similarity-method dice --min-text-similarity-score 0.5 --verbose -O

****************************************
*  Phenotypic Data Model Convert Tool  *
*          - CONVERT-PHENO -           *
*            Version: 0.12             *
*   (C) 2022-2023 Manuel Rueda, CNAG   *
*       The Artistic License 2.0       *
****************************************

==== METHOD: <redcap2bff> ====
Processing: <t/redcap2bff/in/redcap_data.csv>
Writing </content/redcap_individuals.json> file



We'll run again the whole thing, but this time we'll be using Python's interpreter to enable interactive changes in values. We'll be only displaying the items in the array property `measures`. Each individual contains 13 rows.

The code will get executed **automatically** anytime you change a value.

In [35]:
#@title  { run: "auto" }
#@title  { run: "auto", vertical-output: true }
import os
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Extarnal variables
search = 'exact' #@param ["exact", "mixed"] {allow-input: true}
text_similarity_method = 'cosine' #@param ["cosine", "dice"] {allow-input: true}
min_text_similarity_score = 0 #@param {type:"slider", min:0, max:1, step:0.1}

# Creating a dcitionary for args
arg = {
    '-iredcap': 't/redcap2bff/in/Data_table_3TR_IBD_dummydata.csv',
    '-redcap-dictionary': 't/redcap2bff/in/3TRKielTemplateExport01072022_DataDictionary_2022-07-03.csv',
    '--mapping-file': 't/redcap2bff/in/redcap_3tr_mapping.yaml',
    '--search': search,
    '--text-similarity': text_similarity_method,
    '--min-text-similarity-score': min_text_similarity_score
}

# Create CLI string
string = ' '.join([f'{key} {value}' for key, value in arg.items()])
cmd = 'bin/convert-pheno -O' + string

# Run Convert-Pheno
os.system(cmd)

# Import json
df = pd.read_json('/content/redcap_individuals.json')

# Normalize nested data in <measures>
from google.colab.data_table import DataTable
DataTable.max_columns = 30
df = df.explode('measures')
df_norm = pd.json_normalize(df['measures'])
df_norm.head(10)

,date,notes,assayCode.id,assayCode.label,measurementValue.quantity.referenceRange.high,measurementValue.quantity.referenceRange.low,measurementValue.quantity.referenceRange.unit.id,measurementValue.quantity.referenceRange.unit.label,measurementValue.quantity.unit.id,measurementValue.quantity.unit.label,measurementValue.quantity.value,procedure.procedureCode.id,procedure.procedureCode.label
0,1900-01-01,leucocytes /// Field Label=Leucocytes /// Fiel...,NCIT:C51948,Leukocyte Count,200.0,0.0,NCIT:C67242,Cells per Microliter,NCIT:C67242,Cells per Microliter,8.91,NCIT:C27232,Blood Test Result
1,1900-01-01,hemoglobin /// Field Label=Hemoglobin /// Fiel...,NCIT:C64848,Hemoglobin Measurement,20.0,0.0,NCIT:C64783,Gram per Deciliter,NCIT:C64783,Gram per Deciliter,11.20,NCIT:C27232,Blood Test Result
2,1900-01-01,hematokrit /// Field Label=Hematokrit /// Fiel...,NCIT:C64796,Hematocrit Measurement,100.0,0.0,NCIT:C25613,Percentage,NCIT:C25613,Percentage,34.00,NCIT:C27232,Blood Test Result
3,1900-01-01,thrombocytes /// Field Label=Thrombocytes /// ...,NCIT:C51951,Platelet Count,2000.0,0.0,NCIT:C67242,Cells per Microliter,NCIT:C67242,Cells per Microliter,442.00,NCIT:C27232,Blood Test Result
4,1900-01-01,creatinine /// Field Label=Creatinine /// Fiel...,NCIT:C64547,Creatinine Measurement,10000.0,0.0,NCIT:C48508,Micromole per Liter,NCIT:C48508,Micromole per Liter,76.00,NCIT:C27232,Blood Test Result
5,1900-01-01,gfr /// Field Label=GFR CKD-Epi /// Field Note...,NCIT:C90505,Glomerular Filtration Rate,200.0,0.0,NCIT:C98734,Glomerular Filtration Rate Adjusted for BSA,NCIT:C98734,Glomerular Filtration Rate Adjusted for BSA,123.00,NCIT:C27232,Blood Test Result
6,1900-01-01,bilirubin /// Field Label=Bilirubin /// Field ...,NCIT:C38037,Total Bilirubin Measurement,10000.0,0.0,NCIT:C48508,Micromole per Liter,NCIT:C48508,Micromole per Liter,3.50,NCIT:C27232,Blood Test Result
7,1900-01-01,gpt /// Field Label=GPT /// Field Note=xx.x U/...,NCIT:C55098,"Serum Glutamic Pyruvic Transaminase, CTCAE",10000.0,0.0,NCIT:C67376,International Unit per Liter,NCIT:C67376,International Unit per Liter,18.00,NCIT:C27232,Blood Test Result
8,1900-01-01,ggt /// Field Label=gammaGT /// Field Note=xx....,NCIT:C61025,Serum Gamma Glutamyl Transpeptidase Measurement,10000.0,0.0,NCIT:C67376,International Unit per Liter,NCIT:C67376,International Unit per Liter,19.00,NCIT:C27232,Blood Test Result
9,1900-01-01,lipase /// Field Label=Lipase /// Field Note=x...,NCIT:C117748,Lipase Measurement,10000.0,0.0,NCIT:C67376,International Unit per Liter,NCIT:C67376,International Unit per Liter,46.00,NCIT:C27232,Blood Test Result


# Uploading files

It's possible to run jobs using your own files. A few notes:

* The files will be stored in **your instance** of this document. We (developers) don't have access to them.

* Uploaded files will get **deleted** when this runtime is recycled.

* `Convert-Pheno` does not send remote information to our servers and does not store user's information.

* The security is dictated by your Google account / drive.

The files can be uploaded by clicking in the upload button on the left tab of this screen. Files will be uploaded to `/content`.

Once the file is uploaded, you can execute `convert-pheno` command-line as we did above.

Great! I hope this quick tour was useful.

Now it's time to try it out yourself!

Bye!!

Manu